In [ ]:
import os
import subprocess
import time
import shutil
import pandas as pd
import multiprocessing

# Configuration
n_threads = multiprocessing.cpu_count()
os.environ['OMP_NUM_THREADS'] = str(n_threads)
print(f"Using {n_threads} threads")

# Parameters - GRAND OPTIMIZATION
n_steps = 200000
r_restarts = 32
loop_iterations = 10

def run_tree_packer(iteration):
    print(f"\n--- Iteration {iteration}: Running Tree Packer ---")
    # Explicitly set input and output to avoid confusion
    cmd = ["./tree_packer.exe", "-i", "submission.csv", "-o", "submission_v21.csv", "-n", str(n_steps), "-r", str(r_restarts)]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    final_score = None
    for line in process.stdout:
        print(line, end='')
        if "Final:" in line:
            try:
                final_score = float(line.split()[1])
            except:
                pass
                
    process.wait()
    
    if process.returncode != 0:
        print("Tree Packer failed.")
        return False
    
    if os.path.exists("submission_v21.csv"):
        print("Found submission_v21.csv. Updating submission.csv")
        shutil.copy("submission_v21.csv", "submission.csv")
        return True
    else:
        print("No output file found.")
        return False

def run_bp(iteration):
    print(f"\n--- Iteration {iteration}: Running Backward Propagation ---")
    cmd = ["./bp.exe", "submission.csv", "submission_bp.csv"]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    for line in process.stdout:
        print(line, end='')
        
    process.wait()
    
    if process.returncode == 0 and os.path.exists("submission_bp.csv"):
        print("BP finished. Updating submission.csv")
        shutil.copy("submission_bp.csv", "submission.csv")
        return True
    else:
        print("BP failed.")
        return False

# Main Loop
print(f"Starting Grand Optimization Loop: {loop_iterations} iterations")
print(f"Steps: {n_steps}, Restarts: {r_restarts}")

for i in range(loop_iterations):
    print(f"Starting Loop {i+1}/{loop_iterations}")
    
    # Step 1: Tree Packer
    if not run_tree_packer(i+1):
        print("Tree Packer step failed. Stopping.")
        break
        
    # Step 2: BP
    if not run_bp(i+1):
        print("BP step failed. Stopping.")
        break
        
print("Grand Optimization Loop Completed.")
